<a href="https://colab.research.google.com/github/NataliaDiaz/colab/blob/master/Catalyst_DL_Utilities_notebook_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# In google colab, uncomment this:
# ! pip install catalyst==19.03rc0

# Data

In [0]:
import collections
import torch
import torchvision
import torchvision.transforms as transforms


bs = 32
n_workers = 4

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


loaders = collections.OrderedDict()

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True,
    download=True, transform=data_transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=bs,
    shuffle=True, num_workers=n_workers)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False,
    download=True, transform=data_transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=bs,
    shuffle=False, num_workers=n_workers)

loaders["train"] = trainloader
loaders["valid"] = testloader

# Model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Intro
@TODO

In [0]:
# for graphs use `tensorboard --logdir=./logs`

# Setup 1 - typical training

In [0]:
from catalyst.dl.experiments import SupervisedRunner

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_1"

# model, criterion, optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# Setup 2 - training with sheduler

In [0]:
from catalyst.dl.experiments import SupervisedRunner

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_2"

# model, criterion, optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# any Pytorch scheduler supported
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# Setup 3 - training with early stop

In [0]:
from catalyst.dl.experiments import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_3"

# model, criterion, optimizer, scheduler
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        EarlyStoppingCallback(patience=2, min_delta=0.01)
    ],
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# Setup 4 - training with additional metrics

In [0]:
from catalyst.dl.experiments import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback, PrecisionCallback

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_4"

# model, criterion, optimizer, scheduler
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        PrecisionCallback(),
        EarlyStoppingCallback(patience=2, min_delta=0.01)
    ],
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# Setup 5 - training with 1cycle

In [0]:
from catalyst.dl.experiments import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback, PrecisionCallback, OneCycleLR

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_5"

# model, criterion, optimizer, scheduler
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    callbacks=[
        OneCycleLR(cycle_len=n_epochs, div=3, cut_div=4, momentum_range=(0.95, 0.85)),
        PrecisionCallback(),
        EarlyStoppingCallback(patience=2, min_delta=0.01)
    ],
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# Setup 6 - pipeline check

In [0]:
from catalyst.dl.experiments import SupervisedRunner

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_6"

# model, criterion, optimizer, scheduler
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True,
    check=True  # here is the trick
)

# Setup 7 - multi-stage training

In [0]:
from catalyst.dl.experiments import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback, PrecisionCallback

# experiment setup
n_epochs = 10
logdir = "./logs/cifar_simple_notebook_7"

# model, criterion, optimizer, scheduler
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)

# model runner
runner = SupervisedRunner()

# model training - 1
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        PrecisionCallback(),
        EarlyStoppingCallback(patience=2, min_delta=0.01)
    ],
    logdir=logdir,
    n_epochs=n_epochs,
    verbose=True
)

# model training - 2
n_epochs = 5
logdir = "./logs/cifar_simple_notebook_8"
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    logdir=logdir,
    n_epochs=n_epochs
)

# Setup 8 - loader inference

In [0]:
from catalyst.dl.callbacks import InferCallback
loaders = collections.OrderedDict([("infer", loaders["train"])])
runner.infer(
    model=model,
    loaders=loaders,
    callbacks=[InferCallback()],
)

In [0]:
runner.callbacks[0].predictions["logits"].shape

# Setup 9 - batch inference

In [0]:
features, targets = next(iter(loaders["infer"]))

In [0]:
features.shape

In [0]:
runner_in = {runner.input_key: features}
runner_out = runner.predict_batch(runner_in)

In [0]:
runner_out["logits"].shape